In [ ]:
from active_critic.server.analyze import make_acl
import matplotlib.pyplot as plt

In [ ]:
device = 'cpu'
acl, env, expert, seq_len, epsiodes, device = make_acl(device, env_tag='reach', logname='test_plots')
acl.train(epochs=1)

In [ ]:
acl.train_data.reward

In [ ]:
th.all(acl.train_data.obsv[0][1:] == 0)

In [ ]:
acl.policy.eval()
acl.policy.reset()
obs = env.reset()
transitions = sample_expert_transitions(policy=acl.policy.predict, env=env, episodes=1)
attention = acl.policy.critic.attention
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_ylabel('Query')
ax.set_xlabel('Key')
cax = ax.matshow(attention[0].to('cpu'), interpolation='nearest')
fig.colorbar(cax)
plt.show()

In [49]:
import torch as th

def make_step_mask(self, steps, seq_len):
    indices_tensor = th.arange(seq_len).unsqueeze(0)
    mask = indices_tensor < steps[:, None]
    return mask

def proj_actions(self, org_actions: th.Tensor, new_actions: th.Tensor, steps: int):
        with th.no_grad():
            if self.args_obj.take_predicted_action:
                mask = make_step_mask(steps=steps+1, seq_len=org_actions.shape[1])
            else:
                mask = make_step_mask(steps=steps, seq_len=org_actions.shape[1])
            new_actions[mask] = org_actions[mask]
            if self.args_obj.clip:
                th.clamp(new_actions, min=self.clip_min, max=self.clip_max, out=new_actions)
        return new_actions

In [82]:
seq_len = 4
batch_size = 3
dim = 5
current_step = 1
steps = th.tensor([current_step], dtype=int).repeat([batch_size])
mask = make_step_mask(steps=steps, seq_len=seq_len)
a = th.rand([batch_size, seq_len, dim])
b = th.rand([batch_size, seq_len, dim])

In [83]:
steps

tensor([1, 1, 1])

In [84]:
mask

tensor([[ True, False, False, False],
        [ True, False, False, False],
        [ True, False, False, False]])

In [85]:
c = a.clone()
a[mask] = b[mask]

In [86]:
a

tensor([[[0.2145, 0.6745, 0.6704, 0.0344, 0.2557],
         [0.3479, 0.4859, 0.7995, 0.3480, 0.5956],
         [0.5385, 0.4273, 0.0040, 0.2707, 0.1217],
         [0.0308, 0.4225, 0.8689, 0.1896, 0.1826]],

        [[0.3778, 0.8335, 0.5179, 0.3546, 0.6213],
         [0.5252, 0.1670, 0.5183, 0.3185, 0.4178],
         [0.1539, 0.0818, 0.1527, 0.3590, 0.7572],
         [0.0114, 0.8399, 0.9128, 0.0074, 0.8886]],

        [[0.4160, 0.8920, 0.3285, 0.7769, 0.2540],
         [0.7070, 0.8091, 0.9168, 0.2668, 0.6724],
         [0.7314, 0.3905, 0.0933, 0.2044, 0.2339],
         [0.3424, 0.7131, 0.1833, 0.7520, 0.9775]]])

In [87]:
b

tensor([[[0.2145, 0.6745, 0.6704, 0.0344, 0.2557],
         [0.0537, 0.4077, 0.2293, 0.5743, 0.9396],
         [0.3536, 0.1122, 0.3502, 0.0389, 0.6939],
         [0.5271, 0.5581, 0.6695, 0.9640, 0.1137]],

        [[0.3778, 0.8335, 0.5179, 0.3546, 0.6213],
         [0.2030, 0.6157, 0.2384, 0.4694, 0.0954],
         [0.7212, 0.1307, 0.9831, 0.5618, 0.1862],
         [0.0441, 0.0511, 0.5277, 0.1366, 0.5674]],

        [[0.4160, 0.8920, 0.3285, 0.7769, 0.2540],
         [0.4905, 0.2705, 0.1454, 0.8805, 0.1112],
         [0.4741, 0.9272, 0.0636, 0.1069, 0.3819],
         [0.5460, 0.3094, 0.7430, 0.3729, 0.7153]]])